In [1]:
import pandas as pd
import uproot
#import uproot3
import numpy as np
# import track_time_calibration as ttc
from matplotlib import pyplot as plt
import sys
import os 
import fnmatch
sys.path.insert(1, r"C:\Users\nelg\Desktop\Cours\Labo\TP4\Git\utils")
sys.path.insert(1, r"C:\Users\nelg\Desktop\Cours\Labo\TP4\Git\tracking")
from track import Track
from track3D import Track3D
from hit import Hit
from data_loading import *
from parameters import *
from tqdm import tqdm
from track_reconstruction import plot_hits
from physics import dist_line_rect
import pickle
from IPython import display


In [2]:
# Define the paths needed
raw_data_directory = r"C:\Users\nelg\Desktop\Cours\Labo\TP4\raw_data\10h\\" #path to the ecal data
current_directory = os.getcwd()
data_storage = current_directory+r"\extracted_data\\"
runs = None  # List of the runs to analyse. If None, the program will go through all the data available in the raw_data directory
suffix = "" # If one wants to re-run on already analysed data without overwriting the extracted data

In [ ]:
def create_muon_track(df,debug = False):
    candidate_index = []
    low_number = 0
    bottom_touch = 0
    side_touch = 0
    bad_fit = 0
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        hits = [Hit(row,i) for i in range(row['n_hits'])]
        hitsX = [h for h in hits if h.is_sidex]
        hitsY = [h for h in hits if not h.is_sidex]
        
        ## Some events don't have three hits on one of the two sides and are thus not considered
        if len(hitsX) > 3 and len(hitsY) > 3:
            #One only considers the events for which the potential track ended inside the detector
            hitsX.sort(key = lambda hit: -hit.coord[1])
            hitsY.sort(key = lambda hit: -hit.coord[1])
            if hitsX[-1].coord[1] > 1 and hitsY[-1].coord[1] > 1:
                hitsX_last = [hit for hit in hitsX if hit.coord[1]==hitsX[-1].coord[1]]
                hitsY_last = [hit for hit in hitsY if hit.coord[1]==hitsY[-1].coord[1]]
                X_ok = True
                Y_ok = True
                for hit in hitsX_last:
                    if hit.coord[0] == 1 or hit.coord[0] == 24:
                        X_ok = False
                        side_touch += 1
                for hit in hitsY_last:
                    if hit.coord[0] == 1 or hit.coord[0] == 24:
                        Y_ok = False
                        if X_ok:
                            side_touch += 1
                if X_ok and Y_ok:
                    # get track parameters
                    track = Track3D(hits)

                    ## check if track has a "good" chi2 value
                    if track.is_good_2D_fit():
                        candidate_index.append(index)
                    else:
                        bad_fit += 1
            else:
                bottom_touch += 1
        else:
            low_number += 1
    
    return candidate_index, low_number, bottom_touch, side_touch, bad_fit

In [ ]:
# Load the data
files = fnmatch.filter(os.listdir(raw_data_directory), '*.root')

for i,file in enumerate(files):
    if i == 0:
        df = [load_dataset(raw_data_directory+file)]
    else:
        df_i = load_dataset(raw_data_directory+file)
        df_i.index += df[-1].index[-1]+1
        df.append(df_i)

df_hits_total = pd.concat(df)
df_hits = pd.DataFrame.copy(df_hits_total, deep=True)

og_len = len(df_hits_total)

df_hits.query('n_hits > 6', inplace=True)
df_hits.query('n_hits < 50', inplace=True)

new_len = len(df_hits)

print('selected {:.2f}% of all events'.format(new_len/og_len * 100))

In [5]:
## This function finds the indices of event which are good candidate for muon decay : good tracks that don't end on a side of the detector
## have enough hits on both planes, and close to the reconstructed track, and for which the next event is not too long after and has hits
## close to the possible decay point.
def find_muon_decay(df, df_total, time_cutoff = 1500, spacial_cutoff = 4, \
                    save_indices = True, save_hits = False, save_stats = False, \
                    run_name = None, storage_dir = None, \
                    return_stats = True, ):
    """
    Arguments :
        -df : data frame filtered containing only the events with a certain range of n_hits

        -df_total : data frame containing all events

        -time_cutoff : maximal time interval in clock cycles over which a decay is searched

        -spacial_cutoff : maximal distance between the end of the muon track and the potential electron in the next event 

        -save_indices : if True, the indices of the events candidate for muon decay are stored in files with path {storage_dir"events_indices"run_name.txt}
                     
        -save_hits : if True, the lists of hits are stored with pickle in {storage_dir"pickle_events"run_name} for each muon decay event

        -save_stats : if True, the function saves the filtering stats in a dictionary with pickle in {storage_dir"filtering_data"run_name}
        
        -return_stats : if True, the function returns the number of event filtered out at each step of the algorithm

    Returns :
        -candidate_index : indices of the events considered by the algorithm as muon decay

        -time_intervals : time interval in clock cycle between the muon track and the decay for each decay

       The next return numbers are the stats returned if return_stats = True :
       
        -low_number : number of events which contain less that 3 hits in one of the 2 planes

        -bottom_touch : number of events for which the last layer in x or y direction contains a hit

        -side_touch : number of events for which a hit with the lowest z coordinate is a the side of the detector

        -bad_fit : number of events for which the chi square value of the reconstructed track wasn't satisfactory

        -hits_far_from_track : number of hits for which all hits at a distance higher than spacial_cutoff of the reconstructed track (preventive part of the code, shouldn't happen)

        -too_large_time_interval : number of events for which the next event happend too long after to be considered the product of a decay

        -no_spacial_correlation : number of events for which the hits in the next event are far from the end of the track and can thus not be considered
                                 as caused by an electron coming from a decay
    """
    candidate_index = []
    time_intervals = []
    low_number = 0
    bottom_touch = 0
    side_touch = 0
    bad_fit = 0
    too_large_time_interval = 0
    no_spacial_correlation = 0
    hits_far_from_track = 0

    if save_hits:
        decay_data = {'event_index': [], 'track_x0' : [], 'track_tx' : [], 'track_y0' : [], 'track_ty' : [], 'hits_muon': [], 'hits_electron': [], 'time_interval' : []}

    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        hits = [Hit(row,i) for i in range(row['n_hits'])]
        hitsX = [h for h in hits if h.is_sidex]
        hitsY = [h for h in hits if not h.is_sidex]
        
        ## Some events don't have three hits on one of the two sides and are thus not considered
        if len(hitsX) > 3 and len(hitsY) > 3:
            #One only considers the events for which the potential track ended inside the detector
            hitsX.sort(key = lambda hit: -hit.coord[1])
            hitsY.sort(key = lambda hit: -hit.coord[1])
            if hitsX[-1].coord[1] > 1 and hitsY[-1].coord[1] > 1:
                hitsX_last = [hit for hit in hitsX if hit.coord[1]==hitsX[-1].coord[1]]
                hitsY_last = [hit for hit in hitsY if hit.coord[1]==hitsY[-1].coord[1]]
                X_ok = True
                Y_ok = True
                for hit in hitsX_last:
                    if hit.coord[0] == 1 or hit.coord[0] == 24:
                        X_ok = False
                        side_touch += 1
                for hit in hitsY_last:
                    if hit.coord[0] == 1 or hit.coord[0] == 24:
                        Y_ok = False
                        if X_ok:
                            side_touch += 1
                if X_ok and Y_ok:
                    # get track parameters
                    track = Track3D(hits)

                    ## check if track has a "good" chi2 value
                    if track.is_good_2D_fit():
                        next_event = df_total.loc[index+1]
    
                        hits_next_event = [Hit(next_event,i) for i in range(next_event['n_hits'])]
                        hitsX_next_event = [hit for hit in hits_next_event if hit.is_sidex]
                        hitsY_next_event = [hit for hit in hits_next_event if not hit.is_sidex]

                        hitsX = [hit for hit in hitsX if dist_line_rect(track.x.t, track.x.x0, hit.get_pos(), thickness, width) < spacial_cutoff] #Keep only the hits close to the track
                        hitsY = [hit for hit in hitsY if dist_line_rect(track.y.t, track.y.x0, hit.get_pos(), thickness, width) < spacial_cutoff]
                        
                        ## check if there's still hits in the list after removing the ones far from the reconstructed track
                        if len(hitsX) != 0 or len(hitsY) != 0:

                            ## check if the next event happend close enough from the muon track for it to be the product of a decay
                            time_interval = hits_next_event[0].timestamp_event-hits[0].timestamp_event
                            if  time_interval < time_cutoff: 
                                hitsX.sort(key = lambda hit: -hit.get_pos()[1])
                                hitsY.sort(key = lambda hit: -hit.get_pos()[1])

                                X_near = False
                                Y_near = False

                                ## Check if the hits in the next event are close to the end of the muon track
                                if len(hitsX) != 0:
                                    for h in hitsX_next_event:
                                        if np.linalg.norm(h.get_pos()-hitsX[-1].get_pos()) < spacial_cutoff:
                                            X_near = True
                                if len(hitsY) != 0:
                                    for h in hitsY_next_event:
                                        if np.linalg.norm(h.get_pos()-hitsY[-1].get_pos()) < spacial_cutoff:
                                            Y_near = True
                                if X_near and Y_near:
                                    candidate_index.append(index)
                                    time_intervals.append(time_interval)
                                    if save_hits:
                                        decay_data['event_index'].append(index)
                                        decay_data['track_x0'].append(track.x.x0)
                                        decay_data['track_tx'].append(track.x.t)
                                        decay_data['track_y0'].append(track.y.x0)
                                        decay_data['track_ty'].append(track.y.t)
                                        decay_data['hits_muon'].append(hits)
                                        decay_data['hits_electron'].append(hits_next_event)
                                        decay_data['time_interval'].append(time_interval)
                                        
                                else:
                                    no_spacial_correlation += 1
                            else:
                                too_large_time_interval += 1
                        else:
                            hits_far_from_track += 1
                    else:
                        bad_fit += 1
            else:
                bottom_touch += 1
        else:
            low_number += 1
    if save_indices:
        np.savetxt(storage_dir+"events_indices"+run_name+".txt")
    if save_hits:
        
    if save_stats:
        og_len = len(df_hits_total)
        new_len = len(df_hits)
        filtering = pd.DataFrame({'og_len' : [og_len],
                        'new_len' : [new_len],
                        'low_number' : [low_number],
                        'bottom_touch' : [bottom_touch],
                        'side_touch' : [side_touch],
                        'bad_fit': [bad_fit],
                        'too_large_time_interval' : [too_large_time_interval],
                        'hits_far_from_track' : [hits_far_from_track],
                        'no_spacial_correlation' : [no_spacial_correlation]})
        filtering.to_pickle(storage_dir+"filtering_data"+run_name)
    if return_stats:  
        return candidate_index, time_intervals, low_number, bottom_touch, side_touch, bad_fit, hits_far_from_track, too_large_time_interval, no_spacial_correlation
    else:
        return candidate_index, time_intervals

In [ ]:
# Load the data
if runs == None:
    runs = fnmatch.filter(os.listdir(raw_data_directory), '*')

for run in runs:
    files = fnmatch.filter(os.listdir(raw_data_directory), '*.root')

    for i,file in enumerate(files):
        if i == 0:
            df = [load_dataset(raw_data_directory+file)]
        else:
            df_i = load_dataset(raw_data_directory+file)
            df_i.index += df[-1].index[-1]+1
            df.append(df_i)
    
    df_hits_total = pd.concat(df)
    df_hits = pd.DataFrame.copy(df_hits_total, deep=True)
    
    og_len = len(df_hits_total)
    
    df_hits.query('n_hits > 6', inplace=True)
    df_hits.query('n_hits < 50', inplace=True)
    
    new_len = len(df_hits)
    
    print('selected {:.2f}% of all events'.format(new_len/og_len * 100))

    good_indices, low_number, bottom_touch, side_touch, bad_fit, hits_far_from_track, too_large_time_interval, no_spacial_correlation = find_muon_decay(df_hits,df_hits_total)

    np.savetxt(data_storage+"muon_decay_candidates"+run+suffix+".txt",good_indices)

    filtering = pd.DataFrame({'og_len' : [og_len],
                            'new_len' : [new_len],
                            'low_number' : [low_number],
                            'bottom_touch' : [bottom_touch],
                            'side_touch' : [side_touch],
                            'bad_fit': [bad_fit],
                            'too_large_time_interval' : [too_large_time_interval],
                            'hits_far_from_track' : [hits_far_from_track],
                            'no_spacial_correlation' : [no_spacial_correlation]})
    filtering.to_pickle(data_storage+"filtering_data"+run+suffix)


In [ ]:
# This cell allows to check the selected events, as well as the next recorded event (where the muon decay should be observed)
%matplotlib inline
i = 0
kb = ""
while kb == "":
    event = df_hits.loc[candidate_index[i]]
    next_event = df_hits_total.loc[candidate_index[i]+1]

    hits = [Hit(event,i) for i in range(event['n_hits'])]
    hitsX = [h for h in hits if h.is_sidex]
    hits_next_event = [Hit(next_event,i) for i in range(next_event['n_hits'])]
    hitsX_next_event = [h for h in hits_next_event if h.is_sidex]

    track = Track(hitsX)
    fig,ax = plot_hits(hits,True,True,scaling=0.5)
    fig_next_event,ax_next_event = plot_hits(hits_next_event,True,True,scaling=0.5)
    z = np.linspace(0,16)
    x = track.t*z+track.x0
    ax.plot(x,z,'r-')
    display.clear_output(wait=False)
    print(hits_next_event[0].timestamp_event-hits_next_event[0].timestamp-hits[-1].timestamp_event+hits[-1].timestamp)
    display.display(fig)
    display.display(fig_next_event)
    kb = input()
    i = i+1

display.clear_output(wait=False)

In [10]:
filtering = pd.read_pickle(data_storage+"filtering_data"+suffix)
og_len = filtering['og_len'][0]
new_len = filtering['new_len'][0]
low_number = filtering['low_number'][0]
bottom_touch = filtering['bottom_touch'][0]
side_touch = filtering['side_touch'][0]
bad_fit = filtering['bad_fit'][0]
too_large_time_interval = filtering['too_large_time_interval'][0]
hits_far_from_track = filtering['hits_far_from_track'][0]
no_spacial_correlation = filtering['no_spacial_correlation'][0]

In [11]:
# Showcase of the numbers of rejection at each step of the filtering 
print("original length : ", og_len)
print("n_hits between 6 and 50 : ", new_len)
n = new_len
print("\u2937 Too low number of hits in one plane : ", low_number, " over ", n)
n -= low_number
print(" \u2937 Last layer touched : \t\t", bottom_touch, " over ", n)
n -= bottom_touch
print("  \u2937 Possible side exit : \t\t", side_touch, " over ", n)
n -= side_touch
print("   \u2937 Bad fit : \t\t\t\t", bad_fit, " over ", n)
n -= bad_fit
print("    \u2937 Too large time interval : \t", too_large_time_interval, " over ", n)
n -= too_large_time_interval
print("     \u2937 Hits far from track : \t\t", hits_far_from_track, " over ", n)
n -= hits_far_from_track
print("      \u2937 No spacial correlation : \t", no_spacial_correlation, " over ", n)
print("Final number : ", len(good_candidates))

original length :  16032467
n_hits between 6 and 50 :  707799
⤷ Too low number of hits in one plane :  73184  over  707799
 ⤷ Last layer touched : 		 502410  over  634615
  ⤷ Possible side exit : 		 94700  over  132205
   ⤷ Bad fit : 				 28000  over  37505
    ⤷ Too large time interval : 	 6065  over  9505
     ⤷ Hits far from track : 		 0  over  3440
      ⤷ No spacial correlation : 	 2806  over  3440
Final number :  661


In [6]:
good_candidates = np.loadtxt(data_storage+"final_muon_decay_candidates"+suffix+".txt")

In [ ]:
#This cell allows to plot the final candidates for muon decay (original track + next event)
%matplotlib inline

h
i = 0
kb = ""
while kb == "":
    event = df_hits.loc[good_candidates[i]]
    next_event = df_hits_total.loc[good_candidates[i]+1]

    hits = [Hit(event,i) for i in range(event['n_hits'])]
    hitsX = [h for h in hits if h.is_sidex]
    hits_next_event = [Hit(next_event,i) for i in range(next_event['n_hits'])]
    hitsX_next_event = [h for h in hits_next_event if h.is_sidex]

    track = Track(hitsX)
    fig,ax = plot_hits(hits,True,True,scaling = 0.5,hits_next = hits_next_event)
    z = np.linspace(0,16)
    x = track.t*z+track.x0
    ax.plot(x,z,'r-')
    display.clear_output(wait=False)
    print("i = ", i)
    print(hits_next_event[0].timestamp_event-hits_next_event[0].timestamp-hits[-1].timestamp_event+hits[-1].timestamp)
    # fig.savefig("fig_muon_decay_{}.jpg".format(i))
    display.display(fig)
    kb = input()
    i = i+1

display.clear_output(wait=False)

In [14]:
# Compute the time intervals between the muon tracks and the muon decays, and save the data of each decay as a pandas dataframe
import pandas as pd
import pickle
good_candidates = np.loadtxt(data_storage+"muon_decay_candidates"+run+suffix+".txt").astype(int)
time_intervals = []
t_clock_cycle = 6.25

decay_data = {'event_index': [], 'track_x0' : [], 'track_tx' : [], 'track_y0' : [], 'track_ty' : [], 'hits_muon': [], 'hits_electron': [], 'time_interval' : []}

for i in good_candidates:
    event = df_hits.loc[i]
    next_event = df_hits_total.loc[i+1]

    t_muon = event['timestamp_event']
    t_decay = next_event['timestamp_event']
    time_interval = (t_decay-t_muon)*t_clock_cycle

    hits_muon = [Hit(event,i) for i in range(event['n_hits'])]
    hits_electron = [Hit(next_event,i) for i in range(next_event['n_hits'])]
    track = Track3D(hits_muon)

    decay_data['event_index'].append(i)
    decay_data['track_x0'].append(track.x.x0)
    decay_data['track_tx'].append(track.x.t)
    decay_data['track_y0'].append(track.y.x0)
    decay_data['track_ty'].append(track.y.t)
    decay_data['hits_muon'].append(hits_muon)
    decay_data['hits_electron'].append(hits_electron)
    decay_data['time_interval'].append(time_interval)

    time_intervals.append((t_decay-t_muon)*t_clock_cycle)

decay_data = pd.DataFrame.from_dict(decay_data) # translate the dictionary into a pandas dataframe
decay_data.to_pickle(data_storage+"pickle_decay_data"+suffix)

time_intervals = np.array(time_intervals)
np.savetxt(data_storage+"time_intervals"+suffix+".txt",time_intervals)
